In [ ]:
# ==========================================================
# Importando as bibliotecas necessárias
# ==========================================================
import requests
from pyspark.sql import SparkSession  # Para criar e manipular DataFrames no Spark
import pandas as pd


# Criando a sessão Spark
spark = SparkSession.builder \
    .appName("API REST Countries") \
    .getOrCreate()

print("🚀 SparkSession criada com sucesso!")

In [ ]:
# Lendo a tabela para um DataFrame aux_tabela_dados_clima
df_cidades_paises = spark.sql("SELECT capital, nome_resumido FROM view_paises_codigos")

In [ ]:
# Convertendo para lista de tuplas
cidades_paises = [tuple(row) for row in df_cidades_paises.collect()]

# Exibindo o resultado
print(cidades_paises)

In [ ]:
# ==========================================
# chave da API OpenWeatherMap
# ==========================================
api_key = "e0b696d684ea5b8a15f7f141ca979bc3"  

# ==========================================
# Lista de cidades + código do país
# ==========================================
# Filtrando linhas que tenham capital e nome_resumido não nulos ou vazios
cidades_paises = [
    (row['capital'], row['nome_resumido'])
    for row in df_cidades_paises.collect()
    if row['capital'] and row['nome_resumido']
]


# ==========================================
# Lista para armazenar os resultados
# ==========================================
dados_clima = []

# ==========================================
# Consultar cada cidade na API OpenWeatherMap
# ==========================================
for cidade, codigo_pais in cidades_paises:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": f"{cidade},{codigo_pais}",
        "appid": api_key,
        "units": "metric",
        "lang": "pt_br"
    }

    resposta = requests.get(url, params=params)

    if resposta.status_code == 200:
        clima_info = resposta.json()

        dados_clima.append({
            "cidade": cidade,
            "codigo_pais": clima_info['sys']['country'],
            "temperatura": clima_info['main']['temp'],
            "sensacao_termica": clima_info['main']['feels_like'],
            "temperatura_min": clima_info['main']['temp_min'],
            "temperatura_max": clima_info['main']['temp_max'],
            "umidade": clima_info['main']['humidity'],
            "descricao_clima": clima_info['weather'][0]['description']
        })
    else:
        print(f"Erro ao consultar clima para: {cidade}, {codigo_pais} | Status: {resposta.status_code} | Erro: {resposta.text}")

# ==========================================
# Converter para DataFrame
# ==========================================
df_clima = pd.DataFrame(dados_clima)
print(df_clima)
